<a href="https://colab.research.google.com/github/Manos-Mak120/Makrakis-Manos/blob/main/trump_articles_df_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import time
import random

from tqdm import tqdm

In [2]:
def get_webpage_soup(url):
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  response = requests.get(url, headers=headers)
  if response.status_code != 200:
    return None
  else:
    return BeautifulSoup(response.text, 'html.parser')

In [3]:
def get_webpage_soup(url):
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  response = requests.get(url, headers=headers)
  if response.status_code != 200:
    return None
  else:
    return BeautifulSoup(response.text, 'html.parser')
def get_article_elements_from_soup(soup, article_url):
    # title
    try:
        title = soup.find("h1").text.strip()
    except:
        title = ""

    # datetime
    try:
        datetime_str = soup.find("time")["datetime"]
        date_str = datetime_str.split("T")[0]
        time_str = datetime_str.split("T")[1]
    except:
        date_str = ""
        time_str = ""
    # section
    try:
        section_div = soup.find("div", class_="main-category-block") or soup.find("div", class_="main-category")
        section = section_div.text.strip() if section_div else ""
    except:
        section = ""

    # author
    try:
      author_tag = soup.find("a", class_="author-name") or soup.find("span", class_="author-name")
      author = author_tag.text.strip() if author_tag else ""
    except:
        author = ""
    # excerpt
    try:
        excerpt_div = soup.find("div", class_="p.main-intro") or soup.find(itemprop="description") or soup.find("div", class_="nx-excerpt")
        excerpt = excerpt_div.text.strip() if excerpt_div else ""
    except:
        excerpt = ""

    # text body
    try:
      content_div = soup.find("div", class_="main-text story-fulltext")
      elements = content_div.find_all(["p", "h2", "h3", "ul", "ol", "li"]) if content_div else []
      text_body = " ".join(el.get_text(strip=True) for el in elements if el.get_text(strip=True))
    except:
        text_body = ""

    # feature image
    try:
        img_block = soup.find("div", class_="article-image") or soup.find("figure")
        feat_img_link = img_block.find("img")["src"] if img_block and img_block.find("img") else ""
    except:
        feat_img_link = ""

    # image caption
    try:
        caption_div = soup.find("div", class_="image-caption") or soup.find("figcaption")
        image_caption = caption_div.text.strip() if caption_div else ""
    except:
        image_caption = ""

    articleD = {
        "article_url": article_url,
        "title": title,
        "date": date_str,
        "time": time_str,
        "section": section,
        "author": author,
        "excerpt": excerpt,
        "text_body": text_body,
        "feat_img_link": feat_img_link,
        "feat_img_cation": image_caption
    }

    return articleD

In [4]:
def scrape_newsbomb_articles_pages(teasers_df):
  all_articles_dataL = []
  base_url = "https://www.newsbomb.gr"
  for idx, row in tqdm(teasers_df.iterrows(), total=teasers_df.shape[0], desc="Processing rows"):
    article_url_relative = row['article_url']
    article_url = base_url + article_url_relative
    soup = get_webpage_soup(article_url)
    if soup:
     articleD = get_article_elements_from_soup(soup, article_url)
     all_articles_dataL.append(articleD)
    delay = random.uniform(1, 3)
    time.sleep(delay)
  df = pd.DataFrame(all_articles_dataL)
  print(f"\nScraping completed. Fetshed {len(df)} articles.")
  return df

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
trump_teasers_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/trump_df_2025.csv")
trump_teasers_df.head()

,title,article_url,date,time,section,excerpt
0,Πρόεδρος FED για δασμούς Τραμπ: «Πιθανώς θα αυ...,/kosmos/story/1639695/proedros-fed-gia-dasmoys...,04.04.2025,18:39,ΚΟΣΜΟΣ,Τη δική του απάντηση στον Τραμπ έδωσε ο πρόεδρ...
1,Ο Ντόναλντ Τραμπ κάλεσε τον πρόεδρο της Ομοσπο...,/kosmos/story/1639686/o-donalnt-tramp-kalese-t...,04.04.2025,18:14,ΚΟΣΜΟΣ,"Με ανάρτηση στο Truth Social, είπε ότι τώρα θα..."
2,«Σεισμός» στα διεθνή χρηματιστήρια μετά την δι...,/kosmos/story/1639685/seismos-sta-diethni-xrim...,04.04.2025,18:07,ΚΟΣΜΟΣ,Η σημερινή πτώση οφείλεται στο γεγονός ότι «συ...
3,Ο Τραμπ παίζει γκολφ καθώς οι αμερικανικές μετ...,/kosmos/story/1639670/o-tramp-paizei-golf-kath...,04.04.2025,17:30,ΚΟΣΜΟΣ,"Στη Φλόριντα, ο Ντόναλντ Τραμπ απολαμβάνει ένα..."
4,"«Κάρφωσε» την Κίνα ο Τραμπ: «Πανικοβλήθηκε, το...",/kosmos/story/1639656/karfose-tin-kina-o-tramp...,04.04.2025,16:46,ΚΟΣΜΟΣ,Νωρίτερα σήμερα η Κίνα αποφάσισε να ανταποκριθ...


In [8]:
trump_teasers_df_test = trump_teasers_df.head(10)
trump_teasers_df_test

,title,article_url,date,time,section,excerpt
0,Πρόεδρος FED για δασμούς Τραμπ: «Πιθανώς θα αυ...,/kosmos/story/1639695/proedros-fed-gia-dasmoys...,04.04.2025,18:39,ΚΟΣΜΟΣ,Τη δική του απάντηση στον Τραμπ έδωσε ο πρόεδρ...
1,Ο Ντόναλντ Τραμπ κάλεσε τον πρόεδρο της Ομοσπο...,/kosmos/story/1639686/o-donalnt-tramp-kalese-t...,04.04.2025,18:14,ΚΟΣΜΟΣ,"Με ανάρτηση στο Truth Social, είπε ότι τώρα θα..."
2,«Σεισμός» στα διεθνή χρηματιστήρια μετά την δι...,/kosmos/story/1639685/seismos-sta-diethni-xrim...,04.04.2025,18:07,ΚΟΣΜΟΣ,Η σημερινή πτώση οφείλεται στο γεγονός ότι «συ...
3,Ο Τραμπ παίζει γκολφ καθώς οι αμερικανικές μετ...,/kosmos/story/1639670/o-tramp-paizei-golf-kath...,04.04.2025,17:30,ΚΟΣΜΟΣ,"Στη Φλόριντα, ο Ντόναλντ Τραμπ απολαμβάνει ένα..."
4,"«Κάρφωσε» την Κίνα ο Τραμπ: «Πανικοβλήθηκε, το...",/kosmos/story/1639656/karfose-tin-kina-o-tramp...,04.04.2025,16:46,ΚΟΣΜΟΣ,Νωρίτερα σήμερα η Κίνα αποφάσισε να ανταποκριθ...
5,Κατακόκκινη άνοιξε η Wall Street μετά τους δασ...,/kosmos/story/1639651/katakokkini-anoikse-i-wa...,04.04.2025,16:40,ΚΟΣΜΟΣ,Ο μεγαλύτερος χρηματιστηριακός δείκτης - ο S&P...
6,Νέο μήνυμα Τραμπ: Είναι μια καλή στιγμή για να...,/kosmos/story/1639646/neo-minyma-tramp-einai-m...,04.04.2025,16:09,ΚΟΣΜΟΣ,"«Οι πολιτικές μου δεν θα αλλάξουν ποτέ», υπόσχ..."
7,Ρούτε για δασμούς: Οι νέες πολιτικές που ανακο...,/kosmos/story/1639624/royte-oi-nees-politikes-...,04.04.2025,14:42,ΚΟΣΜΟΣ,Ο επικεφαλής του ΝΑΤΟ απέφυγε τεχνηέντως τις ε...
8,Δασμοί Τραμπ και Κίνας: Πάνω από 5% χάνει ο χα...,/kosmos/story/1639618/dasmoi-tramp-kai-kinas-p...,04.04.2025,14:31,ΚΟΣΜΟΣ,Το Πεκίνο λέει ότι οι αμερικανικοί δασμοί στα ...
9,Οι ευρωπαϊκές αγορές κατρακύλησαν μετά την αντ...,/kosmos/story/1639609/oi-evropaikes-agores-kat...,04.04.2025,14:14,ΚΟΣΜΟΣ,Οι παγκόσμιες μετοχές διολίσθησαν για δεύτερη ...


In [9]:
articles_df = scrape_newsbomb_articles_pages(trump_teasers_df)
articles_df

Processing rows: 100%|██████████| 1431/1431 [1:11:49<00:00,  3.01s/it]



Scraping completed. Fetshed 1431 articles.


,article_url,title,date,time,section,author,excerpt,text_body,feat_img_link,feat_img_cation
0,https://www.newsbomb.gr/kosmos/story/1639695/p...,Πρόεδρος FED για δασμούς Τραμπ: «Πιθανώς θα αυ...,2025-04-04,18:39:04+03:00,ΚΟΣΜΟΣ,Newsbomb,Τη δική του απάντηση στον Τραμπ έδωσε ο πρόεδρ...,Ο πρόεδρος τηςFedυπογραμμίζει τον κίνδυνο απογ...,https://img.bbmd.gr/img/1260/max/82/2025/04/04...,INTIME.
1,https://www.newsbomb.gr/kosmos/story/1639686/o...,Ο Ντόναλντ Τραμπ κάλεσε τον πρόεδρο της Ομοσπο...,2025-04-04,18:14:57+03:00,ΚΟΣΜΟΣ,Newsbomb,"Με ανάρτηση στο Truth Social, είπε ότι τώρα θα...",ΟΝτόναλντ Τραμπκάλεσε τον πρόεδρο της Ομοσπονδ...,https://img.bbmd.gr/img/1260/max/82/2025/04/04...,"Ο πρόεδρος της Ομοσπονδιακής Τράπεζας των ΗΠΑ,..."
2,https://www.newsbomb.gr/kosmos/story/1639685/s...,«Σεισμός» στα διεθνή χρηματιστήρια μετά την δι...,2025-04-04,18:07:24+03:00,ΚΟΣΜΟΣ,Newsbomb,Η σημερινή πτώση οφείλεται στο γεγονός ότι «συ...,Με πτώσηάνοιξε και σήμερα το Χρηματιστήριο της...,https://img.bbmd.gr/img/1260/max/82/2025/04/04...,ΑΠΕ-ΜΠΕ
3,https://www.newsbomb.gr/kosmos/story/1639670/o...,Ο Τραμπ παίζει γκολφ καθώς οι αμερικανικές μετ...,2025-04-04,17:30:40+03:00,ΚΟΣΜΟΣ,Newsbomb,"Στη Φλόριντα, ο Ντόναλντ Τραμπ απολαμβάνει ένα...","Ο πρόεδρος των ΗΠΑ,Ντόναλντ Τραμπ, έφτασε στο ...",https://img.bbmd.gr/img/1260/max/82/2025/04/04...,The Associated Press
4,https://www.newsbomb.gr/kosmos/story/1639656/k...,"«Κάρφωσε» την Κίνα ο Τραμπ: «Πανικοβλήθηκε, το...",2025-04-04,16:46:44+03:00,ΚΟΣΜΟΣ,Newsbomb,Νωρίτερα σήμερα η Κίνα αποφάσισε να ανταποκριθ...,"«Η Κίνα το έπαιξε λάθος », διαβεβαιώνει ο Ντόν...",https://img.bbmd.gr/img/1260/max/82/2025/04/04...,Τη δική του απάντηση έδωσε εκ νέου στο Πεκίνο ...
...,...,...,...,...,...,...,...,...,...,...
1426,https://www.newsbomb.gr/kosmos/story/1599378/a...,Αμερικανικές εκλογές 2024: Ο Ντόναλντ Τραμπ μα...,2024-11-06,07:00:04+02:00,ΚΟΣΜΟΣ,Newsbomb,Από την πρώτη στιγμή ο Έλον Μασκ βρίσκεται στο...,ΟΝτόναλντ Τραμπφαίνεται πως είναι ο μεγάλος νι...,https://img.bbmd.gr/img/1260/max/82/2024/11/06...,The Associated Press
1427,https://www.newsbomb.gr/kosmos/story/1599375/e...,Εκλογές ΗΠΑ 2024: Ενισχύονται τα μέτρα ασφαλεί...,2024-11-06,06:55:58+02:00,ΚΟΣΜΟΣ,Newsbomb,Αναπτύσσονται αστυνομικές δυνάμεις σε κρίσιμα ...,Καθώς μπαίνουμε στην τελική ευθεία για το τελι...,https://img.bbmd.gr/img/1260/max/82/2024/11/06...,
1428,https://www.newsbomb.gr/kosmos/story/1599372/e...,Εκλογές ΗΠΑ 2024: Οι Μυστικές Υπηρεσίες ζήτησα...,2024-11-06,06:44:46+02:00,ΚΟΣΜΟΣ,Newsbomb,Το Newsbomb.gr μεταδίδει απευθείας από το Καπι...,Ανταπόκριση από ΗΠΑ / Δημήτρης Φιλιππίδης Tο N...,https://img.bbmd.gr/img/1260/max/82/2024/04/10...,
1429,https://www.newsbomb.gr/oikonomia/story/159937...,Αμερικανικές εκλογές ΗΠΑ: Αγοραστική τρέλα για...,2024-11-06,06:41:06+02:00,ΟΙΚΟΝΟΜΙΑ,Αμαλία Κάτζου,Ο Τραμπ έχει δεσμευτεί να κάνει τις ΗΠΑ την υπ...,Το bitcoin εκτινάχθηκε σε υψηλό ρεκόρ όλων των...,https://img.bbmd.gr/img/1260/max/82/2024/11/06...,Copyright 2024 The Associated Press. All right...


In [10]:
articles_df['text_body'][0]  #έλεγχος ότι πήρε όλο το κείμενο

'Ο πρόεδρος τηςFedυπογραμμίζει τον κίνδυνο απογοήτευσης για την αμερικανική οικονομία Ο πρόεδρος της αμερικανικής κεντρικής τράπεζας (Fed) μίλησε στον Τύπο την Παρασκευή μετά τη διήμερη συνεδρίαση της Ομοσπονδιακής Επιτροπής Ανοικτής Αγοράς για την πολιτική επιτοκίων. Σύμφωνα με τον ίδιο, οι τελωνειακοί δασμοί που επιβλήθηκαν από τον Ντόναλντ Τραμπ«πιθανώς θα αυξήσουν τον πληθωρισμό»και θα υπάρξει κίνδυνος να αυξήσουν την ανεργία και να επιβραδύνουν την ανάπτυξη στις Ηνωμένες Πολιτείες. Οι φόροι στα εισαγόμενα προϊόντα θα είναι«σημαντικά πιο εκτεταμένοι από ό,τι αναμενόταν», γεγονός που«πιθανώς θα αυξήσει τον πληθωρισμό τα επόμενα τρίμηνα»και θα οδηγήσει σε«επιβράδυνση της ανάπτυξης», επιβεβαίωσε ο Jerome Powell στην ομιλία του, αναφέροντας επίσης έναν«αυξημένο κίνδυνο για την απασχόληση». Watch live: Jerome Powell speaks on the economy after Trump announces reciprocal tariffshttps://t.co/xjKIupiZ64 Νωρίτερα,ο Ντόναλντ Τραμπ κάλεσε τον πρόεδρο της Ομοσπονδιακής Τράπεζαςτων ΗΠΑ να μειώσ

In [11]:
articles_df['feat_img_link'][0]

'https://img.bbmd.gr/img/1260/max/82/2025/04/04/seve-dasmoi.jpg?t=TzK4dxglbBKwoEQNehL3JQ'

In [12]:
articles_df = scrape_newsbomb_articles_pages(trump_teasers_df)
articles_df

Processing rows: 100%|██████████| 1431/1431 [1:11:44<00:00,  3.01s/it]



Scraping completed. Fetshed 1431 articles.


,article_url,title,date,time,section,author,excerpt,text_body,feat_img_link,feat_img_cation
0,https://www.newsbomb.gr/kosmos/story/1639695/p...,Πρόεδρος FED για δασμούς Τραμπ: «Πιθανώς θα αυ...,2025-04-04,18:39:04+03:00,ΚΟΣΜΟΣ,Newsbomb,Τη δική του απάντηση στον Τραμπ έδωσε ο πρόεδρ...,Ο πρόεδρος τηςFedυπογραμμίζει τον κίνδυνο απογ...,https://img.bbmd.gr/img/1260/max/82/2025/04/04...,INTIME.
1,https://www.newsbomb.gr/kosmos/story/1639686/o...,Ο Ντόναλντ Τραμπ κάλεσε τον πρόεδρο της Ομοσπο...,2025-04-04,18:14:57+03:00,ΚΟΣΜΟΣ,Newsbomb,"Με ανάρτηση στο Truth Social, είπε ότι τώρα θα...",ΟΝτόναλντ Τραμπκάλεσε τον πρόεδρο της Ομοσπονδ...,https://img.bbmd.gr/img/1260/max/82/2025/04/04...,"Ο πρόεδρος της Ομοσπονδιακής Τράπεζας των ΗΠΑ,..."
2,https://www.newsbomb.gr/kosmos/story/1639685/s...,«Σεισμός» στα διεθνή χρηματιστήρια μετά την δι...,2025-04-04,18:07:24+03:00,ΚΟΣΜΟΣ,Newsbomb,Η σημερινή πτώση οφείλεται στο γεγονός ότι «συ...,Με πτώσηάνοιξε και σήμερα το Χρηματιστήριο της...,https://img.bbmd.gr/img/1260/max/82/2025/04/04...,ΑΠΕ-ΜΠΕ
3,https://www.newsbomb.gr/kosmos/story/1639670/o...,Ο Τραμπ παίζει γκολφ καθώς οι αμερικανικές μετ...,2025-04-04,17:30:40+03:00,ΚΟΣΜΟΣ,Newsbomb,"Στη Φλόριντα, ο Ντόναλντ Τραμπ απολαμβάνει ένα...","Ο πρόεδρος των ΗΠΑ,Ντόναλντ Τραμπ, έφτασε στο ...",https://img.bbmd.gr/img/1260/max/82/2025/04/04...,The Associated Press
4,https://www.newsbomb.gr/kosmos/story/1639656/k...,"«Κάρφωσε» την Κίνα ο Τραμπ: «Πανικοβλήθηκε, το...",2025-04-04,16:46:44+03:00,ΚΟΣΜΟΣ,Newsbomb,Νωρίτερα σήμερα η Κίνα αποφάσισε να ανταποκριθ...,"«Η Κίνα το έπαιξε λάθος », διαβεβαιώνει ο Ντόν...",https://img.bbmd.gr/img/1260/max/82/2025/04/04...,Τη δική του απάντηση έδωσε εκ νέου στο Πεκίνο ...
...,...,...,...,...,...,...,...,...,...,...
1426,https://www.newsbomb.gr/kosmos/story/1599378/a...,Αμερικανικές εκλογές 2024: Ο Ντόναλντ Τραμπ μα...,2024-11-06,07:00:04+02:00,ΚΟΣΜΟΣ,Newsbomb,Από την πρώτη στιγμή ο Έλον Μασκ βρίσκεται στο...,ΟΝτόναλντ Τραμπφαίνεται πως είναι ο μεγάλος νι...,https://img.bbmd.gr/img/1260/max/82/2024/11/06...,The Associated Press
1427,https://www.newsbomb.gr/kosmos/story/1599375/e...,Εκλογές ΗΠΑ 2024: Ενισχύονται τα μέτρα ασφαλεί...,2024-11-06,06:55:58+02:00,ΚΟΣΜΟΣ,Newsbomb,Αναπτύσσονται αστυνομικές δυνάμεις σε κρίσιμα ...,Καθώς μπαίνουμε στην τελική ευθεία για το τελι...,https://img.bbmd.gr/img/1260/max/82/2024/11/06...,
1428,https://www.newsbomb.gr/kosmos/story/1599372/e...,Εκλογές ΗΠΑ 2024: Οι Μυστικές Υπηρεσίες ζήτησα...,2024-11-06,06:44:46+02:00,ΚΟΣΜΟΣ,Newsbomb,Το Newsbomb.gr μεταδίδει απευθείας από το Καπι...,Ανταπόκριση από ΗΠΑ / Δημήτρης Φιλιππίδης Tο N...,https://img.bbmd.gr/img/1260/max/82/2024/04/10...,
1429,https://www.newsbomb.gr/oikonomia/story/159937...,Αμερικανικές εκλογές ΗΠΑ: Αγοραστική τρέλα για...,2024-11-06,06:41:06+02:00,ΟΙΚΟΝΟΜΙΑ,Αμαλία Κάτζου,Ο Τραμπ έχει δεσμευτεί να κάνει τις ΗΠΑ την υπ...,Το bitcoin εκτινάχθηκε σε υψηλό ρεκόρ όλων των...,https://img.bbmd.gr/img/1260/max/82/2024/11/06...,Copyright 2024 The Associated Press. All right...


In [13]:
from google.colab import drive

In [14]:
scrape_newsbomb_articles_pages(trump_teasers_df).to_csv('/content/drive/MyDrive/trump_articles_df_2025.csv', index=False)

Processing rows: 100%|██████████| 1431/1431 [1:11:25<00:00,  2.99s/it]



Scraping completed. Fetshed 1431 articles.
